<a href="https://colab.research.google.com/github/mbalbi/ciencia_de_datos/blob/main/tps/1c2024/Clase_de_apoyo_Funciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [8]:
import pandas as pd
import numpy as np

In [9]:
def calculo_viga(altura, ancho, recubrimiento, Mu, Nu, fc, fy):
    """
    Cálculo de la armadura necesaria en una viga de hormigón armado sometida a flexión según el CIRSOC 201.

    Parámetros:
    altura: Altura total de la viga (m)
    ancho: Ancho de la sección de la viga (m)
    recubrimiento: Recubrimiento de la armadura (m)
    Mu: Momento último de diseño (kNm)
    Nu: Fuerza axial última (kN)
    fc: Resistencia característica del hormigón (MPa)
    fy: Límite elástico del acero (MPa)

    Retorno:
    As: Área de armadura necesaria (cm²)
    Pos: Descripción del tipo de armado (tracción, tracción y compresión)
    """

    # Estimación de la ubicación de la armadura
    d = altura - recubrimiento
    Mus = Mu - Nu * (d - altura / 2)  # Momento en kNm y fuerza en kN

    # Definición de Alpha y Beta
    alpha = 0.85
    if fc <= 30:
        beta = 0.85
    elif 30 < fc <= 58:
        beta = 0.85 - 0.05 * (fc - 30) / 7
    else:
        beta = 0.35

    # Estimación del brazo elástico
    phi = 0.9
    jdc = 0.8 * d
    Cc = Mus / (phi * jdc)  # Cc en kN
    c = Cc / (alpha * beta * fc * ancho)/1000  # c en metros

    # Deformación del acero traccionado
    epsilont = 0.003 * (d - c) / c

    # print(jdc,Cc,c,epsilont)

    if epsilont >= 0.005:
        # Ajuste del brazo elástico
        for _ in range(10):
            jdc = d - beta * c / 2
            Cc = Mus / (phi * jdc)
            c = Cc / (alpha * beta * fc * ancho)/1000

        # print(jdc,Cc,c)

        T = Nu / phi + Cc  # T en kN
        Asnec = (T / fy)*1000  # Área de armadura necesaria en cm²
        Asmin = max(((fc**0.5) / (4 * fy)) * ancho * d, 1.4 * ancho * d / fy) * 1e4  # Área mínima en mm²
        As = max(Asnec, Asmin) / 100  # Convertir a cm²
        print(Asmin,As)
        Pos = "Solo tracción"
        return As, Pos

    elif epsilont < 0.005:
        phi = 0.81379

        c = 3 / 7 * d

        # Ajuste del brazo elástico
        for _ in range(10):
            jdc = d - (beta * c / 2)
            Cc = Mus / (phi * jdc)
            c = Cc / (alpha * beta * fc * ancho)/1000


        Mn = Cc * jdc  # Mn en kNm
        Md = phi * Mn

        # print(jdc,Cc,c,Mn,Md)

        if Md > Mu:
            T = Nu / phi + Cc
            Asnec = (T / fy)*1000
            Asmin = max(((fc**0.5) / (4 * fy)) * ancho * d, 1.4 * ancho * d / fy) * 1e4  # Área mínima en mm²
            As = max(Asnec, Asmin) / 100  # Convertir a cm²
            # print(Asmin,As)
            Pos = "Solo tracción"
            return As, Pos

        else:
            DeltaM = Mus - Mn  # Momento en kNm
            jds = d - recubrimiento
            Cs = DeltaM / jds  # Cs en kN
            T = Nu / phi + Cc + Cs
            Asnec = (T / fy)*1000
            Asmin = max(((fc**0.5) / (4 * fy)) * ancho * d, 1.4 * ancho * d / fy) * 1e4  # Área mínima en mm²
            Ast = max(Asnec, Asmin) / 100  # Convertir a cm²

            epsilons = (0.003 / c) * (c - recubrimiento)
            fs = fy if epsilons >= 0.0021 else fy * epsilons / 0.0021
            Asc = Cs / (fs - alpha * fc)
            As = Ast + Asc
            Pos = "Tracción y compresión"
            return As, Pos



In [10]:
fc = 35  # MPa
fy = 420  # MPa

altura_viga = 0.3  # metros
ancho_viga = 0.2  # metros
recubrimiento_viga = 0.05  # metros
momento_ultimo = 270  # kNm
normal_ultima = 0  # kN

asd, dfg = calculo_viga(altura_viga, ancho_viga, recubrimiento_viga, momento_ultimo, normal_ultima, fc, fy)
# print(f"Área de armadura: {asd:.2f} cm²")
print(f"Área de armadura: {round(asd,2)} cm²")
print(f"Tipo de armado: {dfg}")

Área de armadura: 38.06 cm²
Tipo de armado: Tracción y compresión


In [11]:
# Valores de altura de la viga
altura_viga = [round(i, 3) for i in np.arange(0.2, 0.61, 0.01)]
ancho_viga = [round(i, 3) for i in np.arange(0.2, 0.61, 0.02)]

# Inicializar las listas dentro de DATA
DATA = [[], [], [], []]

for valores_1 in ancho_viga:
    for valores_2 in altura_viga:
        As, Pos = calculo_viga(valores_2, valores_1, recubrimiento_viga, momento_ultimo, normal_ultima, fc, fy)
        DATA[0].append(valores_1)  # Agregar valores_1 a la lista de ancho de viga
        DATA[1].append(valores_2)  # Agregar valores_2 a la lista de altura de viga
        DATA[2].append(As)   # Agregar armadura calculada
        DATA[3].append(Pos)   # Agregar posicion calculada

3.239757976459314 18.019186149346417
3.31018749768669 17.496362016799242
3.3806170189140667 17.00793713132408
3.4510465401414434 16.550192536729902
3.521476061368819 16.119974936445008
3.591905582596196 15.714586531627951
3.662335103823571 15.331700703953741
3.732764625050948 14.969296531838806
3.8031941462783245 14.6256072664246
3.8736236675057008 14.299079313740199
3.408788827405017 18.81531191079503
3.4862613007551313 18.2470167328307
3.563733774105245 17.717512659404946
3.6412062474553593 17.222462284915
3.7186787208054732 16.75820881823497
3.796151194155587 16.32163961095712
3.873623667505701 15.910082595575108
3.951096140855815 15.521226469795375
4.028568614205929 15.153058327199266
4.106041087556043 14.803814314174565
4.183513560906157 14.471940154782738
4.260986034256271 14.156059249438822
3.5496478698597698 19.733874060160257
3.6341632953326215 19.108902860223782
3.718678720805473 18.528616768503078
3.8031941462783245 17.987791265068438
3.8877095717511763 17.48205725494656
3.9

In [12]:
print(DATA[0])

[0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.2, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.22, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.24, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.26, 0.28, 0.28, 0.28, 0.28, 0.28, 0.28, 0.28, 0.28, 0.28, 0.

In [13]:
print(altura_viga)

[0.2, 0.21, 0.22, 0.23, 0.24, 0.25, 0.26, 0.27, 0.28, 0.29, 0.3, 0.31, 0.32, 0.33, 0.34, 0.35, 0.36, 0.37, 0.38, 0.39, 0.4, 0.41, 0.42, 0.43, 0.44, 0.45, 0.46, 0.47, 0.48, 0.49, 0.5, 0.51, 0.52, 0.53, 0.54, 0.55, 0.56, 0.57, 0.58, 0.59, 0.6]
